In [12]:
# Import
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

import os
import random
from shutil import copyfile

import cv2
import matplotlib.pyplot as plt

from ae_models import *
from manage_dataset import *
from visualization import *
from performance import *
from data_augmentation import *

In [7]:
# Constants
HEIGHT = 64
WIDTH = 64
TARGET_SIZE = (HEIGHT, WIDTH)
SPLIT_RATIO = 0.9

FOLDER_IMAGES = "dataset/synthetic_sugarbeet_random_weeds/rgb/"
FOLDER_MASKS = "dataset/synthetic_sugarbeet_random_weeds/gt/"
TARGET_FOLDER = "dataset/synthetic_sugarbeet_random_weeds/train_test/"

BATCH_SIZE = 10
EPOCHS = 1

In [8]:
# Unzip file
import tarfile

filename = 'dataset/synthetic_sugarbeet_random_weeds'

# my_tar = tarfile.open(filename + '.tar.gz')
# my_tar.extractall('dataset')
# my_tar.close()

In [9]:
# Split data

NB_TRAINED_IMAGES = split_data(FOLDER_IMAGES, FOLDER_MASKS, TARGET_FOLDER, SPLIT_RATIO)
print('Number of trained images:', NB_TRAINED_IMAGES)

Number of trained images: 1126


In [10]:
# Understand how the mask is made
mask_path = "dataset/synthetic_sugarbeet_random_weeds/gt/"
list_masks = os.listdir(mask_path)

masks_layer1 = []
masks_layer2 = []
masks_layer3 = []
for i in range(5):
    current_mask = cv2.imread(mask_path + list_masks[i])
    masks_layer1 = np.concatenate([masks_layer1, current_mask[:,:,0].flatten()])
    masks_layer2 = np.concatenate([masks_layer1, current_mask[:,:,1].flatten()])
    masks_layer3 = np.concatenate([masks_layer1, current_mask[:,:,2].flatten()])
    
print(set(masks_layer1))
print(set(masks_layer2))
print(set(masks_layer3))

{0.0, 1.0, 2.0}
{0.0, 1.0, 2.0}
{0.0, 1.0, 2.0}


In [13]:
# Generators
train_image_generator, train_mask_generator = create_train_generator(TARGET_FOLDER,
                                                                     BATCH_SIZE,
                                                                     (HEIGHT, WIDTH),
                                                                     preprocessing_masks)
val_image_generator, val_mask_generator = create_validation_generator(TARGET_FOLDER,
                                                                      BATCH_SIZE,
                                                                      (HEIGHT, WIDTH),
                                                                      preprocessing_masks)

my_train_generator = my_image_mask_generator(train_image_generator, train_mask_generator)
my_val_generator = my_image_mask_generator(val_image_generator, val_mask_generator)

Found 1126 images belonging to 1 classes.
Found 1126 images belonging to 1 classes.
Found 126 images belonging to 1 classes.
Found 126 images belonging to 1 classes.


## FCN-8

In [14]:
# start the FCN8
model = fcn8(HEIGHT, WIDTH, 3, 1)

# print(model.summary())

(None, 64, 64, 1)


## U-Net 

In [16]:
# start the Unet
# model = unet(HEIGHT, WIDTH, 3, 1)

# print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 64, 64, 64)   1792        input_2[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 64, 64, 64)   0           conv2d_13[0][0]                  
__________________________________________________________________________________________________
conv2d_14 (Conv2D)              (None, 64, 64, 64)   36928       activation_1[0][0]               
____________________________________________________________________________________________

In [15]:
# Compile your model here
model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

steps = 1000 / BATCH_SIZE

# Train your model here
history = model.fit(my_train_generator,
                    steps_per_epoch=steps,
                    epochs=EPOCHS,
                    verbose=1,
                    validation_data=my_val_generator,
                    validation_steps=steps)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 100.0 steps, validate for 100.0 steps
 12/100 [==>...........................] - ETA: 26s - loss: 0.6954 - accuracy: 0.4999

KeyboardInterrupt: 

In [ ]:
# PLOT LOSS AND ACCURACY
plot_loss_accuracy(history)

In [ ]:
# Save model
model.save("model.h5")

In [ ]:
# Load model
model.load_weights('weights.h5')

In [ ]:
class_wise_iou, class_wise_dice_score = class_wise_metrics(postprocessing_masks_rgb(example_mask), postprocessing_masks_prediction(pred_image))
print(class_wise_iou)
print(class_wise_dice_score)

In [ ]:
for (img, mask, trash) in my_val_generator:
    
    average_iou = 0.
    average_dice = 0.
    count = 0.
    for cur_img, cur_mask in zip(img, mask):
        count += 1
        
        # Compute prediction
        tensor_img = np.expand_dims(cur_img, axis=0)
        result = model.predict(tensor_img)
        
        # Compute performance
        cls_wise_iou, cls_wise_dice_score = class_wise_metrics(cur_mask, postprocessing_masks_prediction(result[0]))        
        
        average_iou += cls_wise_iou[0]
        average_dice += cls_wise_dice_score[0]
    
    average_iou = average_iou / count
    average_dice = average_dice / count
    
    break

print('Average IoU:', average_iou*100)
print('Average Dice:', average_dice*100)